# Machine learning - Assignment 4 - Classification
____
**Author**: Kemal Cikota

**Course**: Machine learning
____

## Introduction

In this assignment, we were first tasked with answeriing and discussing differences between classification models, such as, Linear Discriminant Analysis (LDA), Quadratic Discriminant Analysis (QDA), and k-nearest neighbor (KNN). The later practical part of this assignment involves implementing using the 'Smarket.csv' dataset which is a financial dataset that containst stock market returns. The classification models will be used to predict market movements. Conclusions and interpretations of the outputs will also be made in this notebook to make it clearer as to how and why certain calculations are computed and what their results indicate.

## Conceptual Questions

**1. Discuss the differences between LDA and QDA in terms of their main assumptions
about classes, decision boundaries, number of samples, and overfitting**



## Practical

Short intro about what is about to happen.

### Load the data and get an overview of the data

In [1]:
import pandas as pd # Never coded in R before but this seems to be the equivalent of library(pandas) in R
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

# load Smarket.csv
boston = pd.read_csv('Smarket.csv')

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)